In [19]:
# !pip install nest_asyncio \
#     langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters \
#     python-pptx==1.0.2 nltk==3.9.1 pymupdf lxml \
#     sentence-transformers IProgress \
#     huggingface_hub ipywidgets \
#     qdrant-client langchain_experimental

# !pip install sentence_transformers datasets pyarrow
# !pip install torch
# !pip install accelerate>=0.26.0
# !pip install transformers
# !pip install wandb



In [2]:

import nest_asyncio

nest_asyncio.apply()

In [3]:
#!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters

In [4]:
#!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml

In [5]:
#!pip install -qU sentence-transformers
#!pip install -qU IProgress


In [6]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

In [7]:
hf_username = getpass.getpass("Enter Your Hugging Face Username: ")


In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
from huggingface_hub import whoami
print(whoami())


{'type': 'user', 'id': '67624d1b57e77fe6e0c87ae5', 'name': 'drewgenai', 'fullname': 'Drew DeMarco', 'email': 'drewgenai@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/L6eLaZmCK4jqW3ZTLYIAR.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'newotken', 'role': 'write', 'createdAt': '2025-02-12T04:11:04.130Z'}}}


In [10]:
!mkdir example_files
!mkdir output

mkdir: cannot create directory ‘example_files’: File exists
mkdir: cannot create directory ‘output’: File exists


In [11]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader

path = "example_files/"
text_loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)

1️⃣ Header-Based Chunking (Title-Based Splitter)
Uses document structure to split on headings, section titles, or patterns.
Works well for structured documents with named assessments, numbered lists, or headers.
Example: If it detects Chronic Pain Adjustment Index (CPAI-10), it groups everything under that title.
2️⃣ Semantic Chunking (Text-Meaning Splitter)
Uses embeddings or sentence similarity to decide where to break chunks.
Prevents splitting mid-context if sentences are closely related.
Example: Groups all related pain-assessment questions into one chunk.

###testingbelow


In [12]:
# !pip install langchain_experimental

In [13]:


# #might need to remove all together - don't think it's working
# !pip install --upgrade langchain langchain-experimental
# !pip install --upgrade langchain-community
# !pip install langchain langchain-experimental langchain-community



In [14]:


from langchain_experimental.text_splitter import SemanticChunker

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

from langchain.embeddings import HuggingFaceEmbeddings
model_id = "Snowflake/snowflake-arctic-embed-m"
embedding_model = HuggingFaceEmbeddings(model_name=model_id)

semantic_splitter = SemanticChunker(embedding_model)

all_documents = text_loader.load()
documents_with_metadata = []



/tmp/ipykernel_76652/2495904805.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_id)


In [15]:
from langchain.schema import Document

for doc in all_documents:
    source_name = doc.metadata.get("source", "unknown")  # Get document source

    # Use SemanticChunker to intelligently split text
    chunks = semantic_splitter.split_text(doc.page_content)

    # Convert chunks into LangChain Document format with metadata
    for chunk in chunks:
        doc_chunk = Document(page_content=chunk, metadata={"source": source_name})
        documents_with_metadata.append(doc_chunk)

##########################new testing below

In [16]:
#training_documents = text_loader.load()
training_documents = documents_with_metadata

In [17]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [18]:
# Define split percentages
train_ratio = 0.7  # 70% training
val_ratio = 0.2    # 20% validation
test_ratio = 0.1   # 10% test

# Calculate index breakpoints
total_docs = len(training_documents)
train_size = int(total_docs * train_ratio)
val_size = int(total_docs * val_ratio)

# Perform the splits
training_split_documents = training_documents[:train_size]
val_split_documents = training_documents[train_size:train_size + val_size]
test_split_documents = training_documents[train_size + val_size:]

# Print sizes to verify
print(f"Training set: {len(training_split_documents)} docs")
print(f"Validation set: {len(val_split_documents)} docs")
print(f"Test set: {len(test_split_documents)} docs")




Training set: 4 docs
Validation set: 1 docs
Test set: 2 docs


In [19]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [22]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [23]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [24]:
import asyncio
import uuid
from tqdm import tqdm

async def process_document(document, n_questions):
    questions_generated = await question_generation_chain.ainvoke({"context": document.page_content, "n_questions": n_questions})

    doc_questions = {}
    doc_relevant_docs = {}

    for question in questions_generated.content.split("\n"):
        question_id = str(uuid.uuid4())
        doc_questions[question_id] = "".join(question.split(".")[1:]).strip()
        doc_relevant_docs[question_id] = [document.metadata["id"]]

    return doc_questions, doc_relevant_docs

async def create_questions(documents, n_questions):
    tasks = [process_document(doc, n_questions) for doc in documents]

    questions = {}
    relevant_docs = {}

    for task in tqdm(asyncio.as_completed(tasks), total=len(documents), desc="Processing documents"):
        doc_questions, doc_relevant_docs = await task
        questions.update(doc_questions)
        relevant_docs.update(doc_relevant_docs)

    return questions, relevant_docs

In [25]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 2)
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 2)
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 2)

Processing documents: 100%|██████████| 2/2 [00:01<00:00,  1.98it/s]


In [26]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)


val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)


train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

In [27]:
# !pip install -qU sentence_transformers datasets pyarrow

In [28]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

In [29]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [30]:
BATCH_SIZE = 10

In [31]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [32]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [33]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [34]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [35]:
EPOCHS = 5

In [36]:
#!pip install wandb

import wandb
wandb.init(mode="disabled")

In [37]:
# !pip install torch
# !pip install accelerate>=0.26.0
# !pip install transformers



In [38]:
#!pip install --upgrade --force-reinstall transformers accelerate torch
#!which python



In [46]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='models/midterm-compare-arctic-embed-m-ft',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
1,No log,No log,1.000000,1.000000,1.000000,1.000000,1.000000,0.333333,0.200000,0.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,No log,No log,1.000000,1.000000,1.000000,1.000000,1.000000,0.333333,0.200000,0.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,No log,No log,1.000000,1.000000,1.000000,1.000000,1.000000,0.333333,0.200000,0.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,No log,No log,1.000000,1.000000,1.000000,1.000000,1.000000,0.333333,0.200000,0.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5,No log,No log,1.000000,1.000000,1.000000,1.000000,1.000000,0.333333,0.200000,0.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [47]:
model.push_to_hub(f"{hf_username}/midterm-compare-arctic-embed-m-ft")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

'https://huggingface.co/drewgenai/midterm-compare-arctic-embed-m-ft/commit/695a90e0d9d4a6ca560a5844c0e5a7cf4c4c74a9'

In [48]:
finetune_embeddings = HuggingFaceEmbeddings(model_name=f"{hf_username}/midterm-compare-arctic-embed-m-ft")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

###testingabove

In [33]:

#!pip install -qU huggingface_hub
#!pip install -qU ipywidgets


In [49]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings


# Load the SentenceTransformer model
#model_id = "Snowflake/snowflake-arctic-embed-m"
model_id = f"{hf_username}/demo-compare-arctic-embed-m-ft"  
embedding_model = HuggingFaceEmbeddings(model_name=model_id)
# model_id = "Snowflake/snowflake-arctic-embed-m"
# embedding_model = HuggingFaceEmbeddings(model_name=model_id)
# model_id = "Snowflake/snowflake-arctic-embed-m-v2.0"
# embedding_model = HuggingFaceEmbeddings(model_name=model_id, model_kwargs={"trust_remote_code": True})


# Load documents into Qdrant
qdrant_vectorstore = Qdrant.from_documents(
    documents_with_metadata,
    embedding_model,
    location=":memory:",  # In-memory for testing
    collection_name="document_comparison",
)

# Create a retriever
qdrant_retriever = qdrant_vectorstore.as_retriever()

Some weights of BertModel were not initialized from the model checkpoint at drewgenai/demo-compare-arctic-embed-m-ft and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# from langchain_core.prompts import ChatPromptTemplate

# RAG_PROMPT = """
# CONTEXT:
# {context}

# QUERY:
# {question}

# You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
# """

# rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

# from langchain_openai import ChatOpenAI

# #openai_chat_model = ChatOpenAI(model="gpt-4o")
# openai_chat_model = ChatOpenAI(model="gpt-4o-mini")

# from operator import itemgetter
# from langchain.schema.output_parser import StrOutputParser

# rag_chain = (
#     {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
#     | rag_prompt | openai_chat_model | StrOutputParser()
# )

In [50]:
from langchain_core.prompts import ChatPromptTemplate
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question.

Return the response in **valid JSON format** with the following structure:

[
    {{
        "Derived Description": "A short name for the matched concept",
        "Protocol_1_Name": "Protocol 1 - Matching Element",
        "Protocol_2_Name": "Protocol 2 - Matching Element"
    }},
    ...
]

### Rules:
1. Only output **valid JSON** with no explanations, summaries, or markdown formatting.
2. Ensure each entry in the JSON list represents a single matched data element from the two protocols.
3. If no matching element is found in a protocol, leave it empty ("").
4. **Do NOT include headers, explanations, or additional formatting**—only return the raw JSON list.
5. It should include all the elements in the two protocols.
6. If it cannot match the element, create the row and include the protocol it did find and put "could not match" in the other protocol column.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

from langchain_openai import ChatOpenAI

#openai_chat_model = ChatOpenAI(model="gpt-4o")
openai_chat_model = ChatOpenAI(model="gpt-4o-mini")

from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | rag_prompt | openai_chat_model | StrOutputParser()
)

In [51]:
question_text = """Between these two files containing protocols, can you find the data elements in each that most likely match the element in the other and output a CSV that lists three columns:

The questions within elements will be similar between the two documents and can be used to match the elements.

1. Derived description from the two documents describing the index/measure/scale.
2. A column for each standard.
3. In the column for each name/version, the data element used to capture that description.

There should only be one row for each scale/index/etc.
The description should not be one of the questions but a name that best describes the similar data elements."""

response_text = rag_chain.invoke({"question": question_text})
# response = rag_chain.invoke({"question": question_text})

In [52]:
import json
import pandas as pd

def parse_rag_output(response_text):
    """Extract structured JSON data from the RAG response."""
    try:
        structured_data = json.loads(response_text)

        # Ensure similarity score is always included
        for item in structured_data:
            item.setdefault("Similarity Score", "N/A")  # Default if missing

        return structured_data
    except json.JSONDecodeError:
        print("Error: Response is not valid JSON.")
        return None

def save_to_csv(data, directory="./output", filename="matching_data_elements.csv"):
    """Save structured data to CSV."""
    if not data:
        print("No data to save.")
        return

    file_path = os.path.join(directory, filename)
    df = pd.DataFrame(data, columns=["Derived Description", "Protocol_1_Name", "Protocol_2_Name"])  # Ensure correct columns
    df.to_csv(file_path, index=False)
    print(f"✅ CSV file saved: {filename}")

# Run the pipeline
structured_output = parse_rag_output(response_text)
save_to_csv(structured_output)


✅ CSV file saved: matching_data_elements.csv


In [40]:
# rag_chain.invoke({"question" : "Based on the types of questions asked under each heading.  can you identify the headings in one document that most closely match the second document.  list them e.g   paincoping/doc1  painstrategy/doc2"})

In [41]:
# rag_chain.invoke({"question" : "Based on the types of questions asked under each heading.  can you identify the headings in one document that most closely match the second document.  list them e.g   paincoping/doc1  painstrategy/doc2. these are example headings not the ones in the actual documents.  just list the matches not the rational.  Can you list multiple matches?"})